In [3]:
import sys
!{sys.executable} -m pip install py-algorand-sdk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 KB 457.3 kB/s eta 0:00:001m448.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 KB 2.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 KB 2.3 MB/s eta 0:00:00 MB/s eta 0:00:01


In [2]:
from algosdk import account, mnemonic
from algosdk import transaction

# Create Account

## Account 1

In [3]:
private_key, address = account.generate_account()
print(f"address: {address}")
print(f"private key: {private_key}")
print(f"mnemonic: {mnemonic.from_private_key(private_key)}")

address: JEXLOIOFTCDGK5HIS5PNZVYXJ35VA2ATDYQSCFJVCDF3AIJW4MXVANIYOQ
private key: 0cDn4IGErCj6qXqLgQBHFBd3mOR1jLq0agHUCpoa+uJJLrchxZiGZXTol17c1xdO+1BoEx4hIRU1EMuwITbjLw==
mnemonic: boss pair vacant elite flip mechanic exist kingdom cousin acoustic castle image until champion quality midnight nuclear print absent pull cross mammal blind abstract client


## Account 2

In [12]:
private_key2, address2 = account.generate_account()
print(f"address2: {address2}")
print(f"private key2: {private_key2}")
print(f"mnemonic2: {mnemonic.from_private_key(private_key2)}")

address2: WDHH2DED4PQM4N63W2W67RGFO7XCE4X36RKNFVOYMN7DIPAGHLVYVO5BRI
private key2: qOdX400Uj7ij+jV1x1g8H2YM1VWUsf54hRa0yGV8PUewzn0Mg+Pgzjfbtq3vxMV37iJy+/RU0tXYY340PAY66w==
mnemonic2: vital garbage vast dynamic bullet until steak curious upgrade glide owner giggle boat inspire pencil ranch cabin bicycle area mercy slide gallery delay able abstract


In [9]:
from utils import get_accounts, get_algod_client

# Connect to Client

In [7]:
from algosdk.v2client import algod

algod_address = "http://localhost:4001"
algod_token = "a" * 64
algod_client = algod.AlgodClient(algod_token, algod_address)

# Or, if necessary, pass alternate headers

# Create a new client with an alternate api key header
special_algod_client = algod.AlgodClient(
    "", algod_address, headers={"X-API-Key": algod_token}
)

In [10]:
algod_client = get_algod_client()
accts = get_accounts()

acct1 = accts.pop()
private_key, address = acct1.private_key, acct1.address
address

'XAGM3TH46HGKMEHTZ6EQ33OHC6ZWKDWE5AIS4UFLZIRDIRJNVNXNABA3XQ'

In [11]:
acct2 = accts.pop()
address2 = acct2.address
address2

'LDTQ76VT6H6FEEVDX7BQQ3CCEG2B3KJ7PVXOV47UTR5MEECJQPHJ4T2KFI'

# Check Balance

In [17]:
from typing import Dict, Any

account_info: Dict[str, Any] = algod_client.account_info(address2)
print(f"Account balance: {account_info.get('amount')} microAlgos")

Account balance: 1000000 microAlgos


# Make First Transaction

In [14]:
params = algod_client.suggested_params()
unsigned_txn = transaction.PaymentTxn(
    sender=address,
    sp=params,
    receiver=address2,
    amt=1000000,
    note=b"Hello World",
)
unsigned_txn

# Sign Transaction

In [15]:
# sign the transaction
signed_txn = unsigned_txn.sign(private_key)
signed_txn

# Submit the Transaction

In [16]:
import json
from base64 import b64decode
# submit the transaction and get back a transaction id
txid = algod_client.send_transaction(signed_txn)
print("Successfully submitted transaction with txID: {}".format(txid))

# wait for confirmation
txn_result = transaction.wait_for_confirmation(algod_client, txid, 4)

print(f"Transaction information: {json.dumps(txn_result, indent=4)}")
print(f"Decoded note: {b64decode(txn_result['txn']['txn']['note'])}")

Successfully submitted transaction with txID: FVAB36ZGG5I3C4QBCSJZBS3D2W5W35V3C66IDWEEEFFTKF7LU2CA
Transaction information: {
    "confirmed-round": 1,
    "pool-error": "",
    "txn": {
        "sig": "cDzs9xxqqHgx5PViFwaOn9D6MlpZyTWYwwjA8xi2WcxR5i+FtGbq7eE/ndFthyaZrl9S1ousK4jtxQHeFzPiBA==",
        "txn": {
            "amt": 1000000,
            "fee": 1000,
            "gen": "sandnet-v1",
            "gh": "1+gug8q0BheTJqq6UKA1Z6xmwB51BJ52F8OBfrMpWGw=",
            "lv": 1000,
            "note": "SGVsbG8gV29ybGQ=",
            "rcv": "WDHH2DED4PQM4N63W2W67RGFO7XCE4X36RKNFVOYMN7DIPAGHLVYVO5BRI",
            "snd": "XAGM3TH46HGKMEHTZ6EQ33OHC6ZWKDWE5AIS4UFLZIRDIRJNVNXNABA3XQ",
            "type": "pay"
        }
    }
}
Decoded note: b'Hello World'
